In [3]:
import pandas as pd
from ast import literal_eval
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

In [4]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\BAAMARA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\BAAMARA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\BAAMARA\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\BAAMARA\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [5]:
sw = stopwords.words('english') # List of predefined english  stopwords to be used for computing
lemm = WordNetLemmatizer() # We now define the functions below connecting these imported packages

In [6]:
# todo remove punctuation python

In [7]:
data = pd.read_excel('reviews.xlsx', sheet_name='Sheet1')

In [8]:
data.head()

,Unnamed: 0.1,Unnamed: 0,hotel_name,id_review,title,date,location,user_name,user_link,date_of_stay,rating,review,rating_review,n_review_user,n_votes_review,score_lang,is_bad_review
0,0,0,AC Hotel by Marriott Nice,858676765,The best in nice,Sep 6,"Burbank, California",armenianburbank,/Profile/armenianburbank,August 2022,4,The best thing about this hotel was the manage...,4,34,21,en,1
1,1,1,AC Hotel by Marriott Nice,852510704,Great service,Aug 2022,"L'Orignal, Ontario, Canada",Steve C,/Profile/P6582SZstevec,August 2022,5,The checkin was easy and swift. Breakfast had ...,5,23,27,en,0
2,2,2,AC Hotel by Marriott Nice,855890988,"Good location, unclean room",Aug 2022,"Amsterdam, The Netherlands",Tracy B,/Profile/TryBry,August 2022,3,"Pros: Location, black out curtains, friendly s...",3,1,1,en,1
3,3,3,AC Hotel by Marriott Nice,855208450,No beach beds,Aug 2022,Disneyland Paris,Kiki Y,/Profile/77kikiy,August 2022,2,The advertisement that they have beachbeds ava...,2,26,22,en,1
4,4,4,AC Hotel by Marriott Nice,854313007,"Unclean room, terrible customer service",Aug 2022,NaN,Jac W,/Profile/464jacw,July 2022,1,Had an incredibly frustrating experience and p...,1,1,1,en,1


In [9]:
data.hotel_name.value_counts()

Hotel Le Six                            500
Hotel Relais Bosquet Paris              500
Hotel L'Elysee Val D'Europe             500
Hotel La Comtesse                       500
Hotel Signature St Germain des Pres     500
                                       ... 
Villa Les Hydrangeas                     20
Le Celtique Hotel & Spa                  10
Kyriad Nice - Stade                      10
Hotel Roi Soleil Prestige Strasbourg     10
Hotel Les Volets Rouges - Cassis         10
Name: hotel_name, Length: 88, dtype: int64

In [10]:
data.shape

(31880, 17)

In [11]:
data['description'] = data.apply(lambda row: str(row.title) + str(row.review), axis=1)

In [12]:
def clear_description(description):

    # Dividing the texts into small tokens (sentences into words)
    description = description.lower()
    description_tokens=word_tokenize(description)


    filtered_sen = {w for w in description_tokens if not w in sw}
    f_set=set()
    for fs in filtered_sen:
        f_set.add(lemm.lemmatize(fs))

    return f_set

In [13]:
def comput_similarity(description, input):

    input_tokenize = clear_description(input)
    similarity = description.apply(lambda x:len(input_tokenize.intersection(x)) )

    return  similarity

In [14]:
def recommandation(input,top=10):

    similarity = comput_similarity(data.description_tokenize, input)
    data["similarity"] = similarity
    data.sort_values(by=['similarity', 'review'], ascending=False, inplace=True)
    data.drop_duplicates('hotel_name',keep='first')

    return data.head(top)

In [15]:
description_tokenize = data.description.apply(lambda x:clear_description(x))

In [16]:
data["description_tokenize"] = description_tokenize

In [17]:
col = ['hotel_name', 'title', 'review', 'rating_review', "description_tokenize"] # todo select subset of columns

In [18]:
input = 'good location free wifi spa'

In [21]:
data = data[col].copy()

In [22]:
recommandation( input)

,hotel_name,title,review,rating_review,description_tokenize,similarity
1943,Auberge de Cassagne & Spa,dissapointed,"i had good expectations for this hotel,unfortu...",2,"{view, located, separate, cheese, met, parc, a...",5
12962,Hotel Gounod Nice,A good choice in Nice.,"This 3-star hotel, is an pleasant option for N...",3,"{located, creates, promanade, access, ”, ’, on...",5
30733,Splendid Hotel & Spa,Explorario,The Bap fam are just about to leave (23aug)the...,4,"{view, accommodated, meal, access, 30, hour, n...",5
30704,Splendid Hotel & Spa,Great location but average hotel,My husband and I recently stayed 3 nights at t...,3,"{kept, another, polite, manager, conversation,...",5
7415,Hotel Boscolo Nice,Disappointing.,"An overall poor experience here. First, the go...",2,"{located, anglais.however, negresco, asked, co...",5
21707,Hotel Palm - Astotel,"This should be a benchmark for all hotels , fa...","we picked this hotel based on reviews , price ...",5,"{right, usefull, maybe, arrive, 10mins, access...",4
16299,Hotel Le Mareuil,nice place to stay in Paris,"we had three nice room, nice welcome, friendly...",4,"{three, pariswe, stay, place, hotel, bathroom,...",4
28485,NH Nice,Great Hotel ! Value for money!,i stayed at the hotel for 3 nights on April fo...,5,"{wifi, restaurant, basic, bar, stay, mainly, a...",4
10493,Hotel de Provence,Cosy Hotel,When you visit Cannes and don't wish to pay 30...,4,"{right, take, cheese, boulevard, outside, saw,...",4
28171,NH Nice,Could give 5* but the wifi wasnt working at all,What I love about this hotel:The room is moder...,4,"{located, see, asked, guardian, access, app, t...",4


In [23]:
import gradio as gr

In [27]:


def greet(name):
    return "Hello " + name + "!"

demo = gr.Interface(
    fn=greet,
    inputs=gr.Textbox(lines=2, placeholder="Name Here..."),
    outputs="text",
)
demo.launch()

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


In [ ]:
demo = gr.Interface(
    fn=recommandation,
    inputs=gr.Textbox(lines=2, placeholder="Name Here..."),
    outputs="text",
)
demo.launch()